<a href="https://colab.research.google.com/github/mgnarag/binarization-lowres/blob/main/200Metric_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2

In [8]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            #im1 = im1/4
            data.append(im1)
    return data

def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr


In [9]:
gt = Image.open(file_path+ '4 Testing/ground_truth_200dpi.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth_200dpi.png')
gt = np.asarray(gt) > 128
gt = ~gt

filename = "nbs_200dpi.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
dirty = ImageOps.grayscale(Image.open(directory))
dirty = np.array(dirty)
dirty = Image.fromarray(normalize(dirty).astype('uint8'))

w_dirty, h_dirty = dirty.size


METRIC = []
channel = ['gray','red','green', 'blue']
for model in range(5,7):
  for color in range(0,4):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/200 DPI/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128
      if model == 7:
          n_size = 256
      if model == 8:
          n_size = 256

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          im1 = dirty.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          y = np.where(col > 0.5,1,0) #round the values
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_'+str(channel[color]) + '.png')

      output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png').convert('L')  # convert to grayscale
      img_out = cv2.imread(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png')

      output = np.asarray(output) > 128 # Convert to binary images
      output = ~output

      tp = np.sum(output & gt)
      fp = np.sum(output & ~gt)
      fn = np.sum(~output & gt)
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1score = (2*precision*recall)/(precision + recall)

      I_1 = output & gt
      U_1 = output | gt
      I = np.sum(output & gt)
      U = np.sum(output | gt)
      i_o_u = I/U

      psnr = cv2.PSNR(img_out, img_gt)

      METRIC.append([model, str(channel[color]), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  output_red = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[0]) +  '.png').convert('L'))>128)
  output_green = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))>128)
  output_blue = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))>128)
  output_rgb = ~output_red | ~output_green | ~output_blue

  reconstructed = Image.fromarray(~output_rgb)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB'), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB plus grayscale------------------------------------------------
  print('Model = ', model, '', str('RGB+gray'))
  output_gray = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model) +  '_gray.png').convert('L'))>128)
  output = output_rgb | ~output_gray

  reconstructed = Image.fromarray(~output)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB_GRAY.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB+gray'),precision,recall,f1score,i_o_u,psnr])


METRIC = pd.DataFrame(METRIC)

headers = ['Model','Channel','Precision', 'Recall', 'F1 Score', 'IOU','PSNR' ]
METRIC.columns = headers

METRIC.to_csv(file_path + 'Metric_RGB_.csv')
print(METRIC)

Model =  5  gray
Model =  5  red
Model =  5  green
Model =  5  blue
Model =  5  RGB
Model =  5  RGB+gray
Model =  6  gray
Model =  6  red
Model =  6  green
Model =  6  blue
Model =  6  RGB
Model =  6  RGB+gray
    Model   Channel  Precision    Recall  F1 Score       IOU       PSNR
0       5      gray   0.808590  0.651464  0.721572  0.564422  15.324708
1       5       red   0.798135  0.741815  0.768946  0.624624  15.846793
2       5     green   0.827548  0.620642  0.709315  0.549565  15.273685
3       5      blue   0.860802  0.359103  0.506788  0.339395  13.893677
4       5       RGB   0.741663  0.795499  0.767638  0.622900  15.511463
5       5  RGB+gray   0.741663  0.795499  0.767638  0.622900  15.511463
6       6      gray   0.858807  0.598445  0.705367  0.544839  15.349034
7       6       red   0.778587  0.696948  0.735509  0.581664  15.337708
8       6     green   0.875652  0.590120  0.705075  0.544491  15.403776
9       6      blue   0.972279  0.271071  0.423946  0.268992  13.66558

In [ ]:
TP = output & gt
TP = Image.fromarray(~TP)
plt.imshow(TP,cmap = 'gray')
TP.save(file_path + "5 Tested/TP.png")

FP = output & ~gt
FP = Image.fromarray(~FP)
plt.imshow(FP,cmap = 'gray')
FP.save(file_path + "5 Tested/FP.png")

FN = ~output & gt
FN = Image.fromarray(~FN)
plt.imshow(FN,cmap = 'gray')
FN.save(file_path + "5 Tested/FN.png")